In [39]:
import utils
inp = utils.get_input(2019, 11)[:-1]
print(inp[:96])

Found cached input for 2019 day 11
3,8,1005,8,361,1106,0,11,0,0,0,104,1,104,0,3,8,102,-1,8,10,101,1,10,10,4,10,108,0,8,10,4,10,1001


In [2]:
from collections import namedtuple

coord = namedtuple('coord', ['x', 'y', 'z'])
a = coord(1,2,3)
b = coord(4,5,6)
m = {0: (a, b)}
from copy import deepcopy
n = deepcopy(m)

m == n

True

## machine

In [2]:
import operator
from functools import partial
from itertools import repeat, chain
from collections import namedtuple, defaultdict


class IntCode:
    Op = namedtuple('Op', ['func', 'params', 'resolve_last_as_ptr'])

    def __init__(self, program, pointer=0, rel_base=0, **kwargs):
        self._mem = defaultdict(int, enumerate(program))
        self.ptr = pointer
        self.rel = rel_base
        self.state = "started"
        self.output = None
        self._kwargs = kwargs
        self._ops = {
            1: IntCode.Op(partial(IntCode._math, operator.add), 3, True),
            2: IntCode.Op(partial(IntCode._math, operator.mul), 3, True),
            3: IntCode.Op(IntCode._inp, 1, True),
            4: IntCode.Op(IntCode._out, 1, False),
            5: IntCode.Op(partial(IntCode._jump, lambda x: x != 0), 2, False),
            6: IntCode.Op(partial(IntCode._jump, lambda x: x == 0), 2, False),
            7: IntCode.Op(partial(IntCode._math, operator.lt), 3, True),
            8: IntCode.Op(partial(IntCode._math, operator.eq), 3, True),
            9: IntCode.Op(IntCode._base, 1, False),
            99: IntCode.Op(IntCode._halt, 0, False),
        }

    @property
    def mem(self):
        return list(self._mem.values())

    def run(self, inputs=None):
        inputs = iter(inputs or [])

        while self.state != "halted":
            instruction = self._mem[self.ptr]
            opcode, modes = self._opcode(instruction), self._modes(instruction)
            op = self._ops[opcode]
            params_raw = [self._mem[i] for i in range(self.ptr + 1, self.ptr + op.params + 1)]
            params = self.resolve(params_raw, modes, op.resolve_last_as_ptr)

            orig_ptr = self.ptr
            self._mem, self.ptr, self.rel, output, self.state = \
                op.func(self._mem, self.ptr, self.rel, *params, inputs=inputs, **self._kwargs)
            self.output = output if output is not None else self.output

            if self._kwargs.get('_debug', False):
                print(f"{instruction},{','.join(map(str, params_raw))}",
                      f"-> {opcode:2d} {modes} {params}",
                      f"-> {output, self.ptr, self.rel}")

            if (output is not None) or (self.state == "blocked"):
                yield self

            if self.ptr == orig_ptr:
                self.ptr = self.ptr + op.params + 1
        yield self

    def resolve(self, params, modes, resolve_last_as_ptr):
        def _resolve(p, m, as_ptr):
            if int(m) == 0:
                return self._mem[p] if not as_ptr else p
            elif int(m) == 2:
                return self._mem[self.rel + p] if not as_ptr else self.rel + p
            return p
        resolve_as_ptr = chain(repeat(False, len(params) - 1), [resolve_last_as_ptr])
        return list(map(lambda t: _resolve(*t), zip(params, modes, resolve_as_ptr)))

    @staticmethod
    def _opcode(instruction):
        return int(str(instruction)[-2:])

    @staticmethod
    def _modes(instruction):
        return f"{instruction:05d}"[:3][::-1]
    
    # operations
    
    @staticmethod
    def _math(func, mem, ptr, rel, a, b, out, **kwargs):
        mem[out] = int(func(a, b))
        return mem, ptr, rel, None, "running"

    @staticmethod
    def _inp(mem, ptr, rel, out, inputs, **kwargs):
        try:
            mem[out] = int(next(inputs))
            return mem, ptr, rel, None, "running"
        except StopIteration:
            return mem, ptr, rel, None, "blocked"

    @staticmethod
    def _out(mem, ptr, rel, val, **kwargs):
        return mem, ptr, rel, val, "running"

    @staticmethod
    def _jump(func, mem, ptr, rel, cond, val, **kwargs):
        return mem, (val if func(cond) else ptr), rel, None, "running"

    @staticmethod
    def _base(mem, ptr, rel, val, **kwargs):
        return mem, ptr, (rel + val), None, "running"
    
    @staticmethod
    def _halt(mem, ptr, rel, *args, **kwargs):
        return mem, ptr, rel, None, "halted"


In [64]:
from collections import namedtuple


Coords = namedtuple('Coords', ['x', 'y'])
Vector = namedtuple('Vector', ['coords', 'direction'])
Panel = namedtuple('Panel', ['color', 'painted'])

def turn_and_move(curr, turn_to):
    if curr.direction == '^':
        if turn_to == 0:
            return Vector(Coords(curr.coords.x - 1, curr.coords.y), '<')
        elif turn_to == 1:
            return Vector(Coords(curr.coords.x + 1, curr.coords.y), '>')
    if curr.direction == '<':
        if turn_to == 0:
            return Vector(Coords(curr.coords.x, curr.coords.y + 1), 'v')
        elif turn_to == 1:
            return Vector(Coords(curr.coords.x, curr.coords.y - 1), '^')
    if curr.direction == 'v':
        if turn_to == 0:
            return Vector(Coords(curr.coords.x + 1, curr.coords.y), '>')
        elif turn_to == 1:
            return Vector(Coords(curr.coords.x - 1, curr.coords.y), '<')
    if curr.direction == '>':
        if turn_to == 0:
            return Vector(Coords(curr.coords.x, curr.coords.y - 1), '^')
        elif turn_to == 1:
            return Vector(Coords(curr.coords.x, curr.coords.y + 1), 'v')


panels = defaultdict(lambda: Panel(0, 0))

curr = Vector(Coords(0, 0), '^')
panels[curr.coords] = Panel(1, 0)

program = list(map(int, inp.split(',')))
inputs = [panels[curr.coords].color]

vm = IntCode(program)
vm_gen = vm.run(inputs)

while True:
    try:
        vm = next(vm_gen)
        clr = vm.output

        vm = next(vm_gen)
        vec = vm.output

        panels[curr.coords] = Panel(clr, 1)
        curr = turn_and_move(curr, vec)

        inputs.append(panels[curr.coords].color)
    except StopIteration:
        break


In [81]:
# print(panels)
painted = [k for k, v in panels.items() if v.color and v.painted]

xs = list(map(lambda p: p.x, painted))
ys = list(map(lambda p: p.y, painted))

img = []

for y in range(min(ys), max(ys) + 1):
    row = []
    for x in range(min(xs), max(xs) + 1):
        row.append('#' if Coords(x, y) in painted else ' ')
    img.append(''.join(row))


print('\n'.join(img))


#    ###  #### ####  ##   ##  #### ####
#    #  #    # #    #  # #  # #    #   
#    #  #   #  ###  #    #    ###  ### 
#    ###   #   #    #    # ## #    #   
#    # #  #    #    #  # #  # #    #   
#### #  # #### ####  ##   ### #    ####


In [68]:
# LRZECGFE

In [69]:
sum([v.painted for k, v in panels.items()])

249